In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cafa-5-protein-function-prediction/sample_submission.tsv
/kaggle/input/cafa-5-protein-function-prediction/IA.txt
/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset.fasta
/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset-taxon-list.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/train_sequences.fasta
/kaggle/input/cafa-5-protein-function-prediction/Train/train_taxonomy.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/go-basic.obo
/kaggle/input/2022-06-22/id_seq_term_tax.csv


In [2]:
#seq_file에서 dataframe 형식으로 data 불러오기
seq_file = "/kaggle/input/cafa-5-protein-function-prediction/Train/train_sequences.fasta"
import pandas as pd
import re
def read_fasta(file_path, columns) :
    from Bio.SeqIO.FastaIO import SimpleFastaParser 
    with open(file_path) as fasta_file :  
        records = [] # create empty list

        for title, sequence in SimpleFastaParser(fasta_file):  #SimpleFastaParser Iterate over Fasta records as string tuples. For each record a tuple of two strings is returned: the FASTA title line (without the leading ‘>’ character),  and the sequence (with any whitespace removed). 
            record = []
            title_splits=re.findall(r"[\w']+", title) # Data cleaning is needed
            record.append(title_splits[0])  #First values are ID (Append adds element to a list)
            record.append(len(sequence)) #Second values are sequences lengths
            sequence = " ".join(sequence) #It converts into one line
            record.append(sequence)#Third values are sequences
            records.append(record)
    return pd.DataFrame(records, columns = columns) #We have created a function that returns a dataframe

#Now let's use this function by inserting in the first argument the file name (or file path if your working directory is different from where the fasta file is)        
#And in the second one the names of columns
data = read_fasta(seq_file, columns=["id","sequence_length", "sequence"])
data

,id,sequence_length,sequence
0,P20536,218,M N S V T V S H A P Y T I T Y H D D W E P V M ...
1,O73864,354,M T E Y R N F L L L F I T S L S V I Y P C T G ...
2,O95231,258,M R L S S S P P R G P Q Q L S S F G S V D W L ...
3,A0A0B4J1F4,415,M G G E A G A D G P R G R V K S L G L V F E D ...
4,P54366,415,M V E T N S P P A G Y T L K R S P S D L G E Q ...
...,...,...,...
142241,A0A286YAI0,450,M E T E V D D F P G K A S I F S Q V N P L Y S ...
142242,A0A1D5NUC4,643,M S A A A S A E M I E T P P V L N F E E I D Y ...
142243,Q5RGB0,448,M A D K G P I L T S V I I F Y L S I G A A I F ...
142244,A0A2R8QMZ5,459,M G R K K I Q I T R I M D E R N R Q V T F T K ...


In [3]:
#index 1번의 id에 담겨있는 나머지 정보 불러오기
from Bio import SeqIO

# Open the FASTA file
fasta_sequences = SeqIO.parse(open(seq_file),'fasta')

# Loop over each sequence and print its ID and length
for fasta in fasta_sequences:
    print(fasta)
    print("Sequence ID:", fasta.id)
    print("Sequence length:", len(fasta))
    break

ID: P20536
Name: P20536
Description: P20536 sp|P20536|UNG_VACCC Uracil-DNA glycosylase OS=Vaccinia virus (strain Copenhagen) OX=10249 GN=UNG PE=1 SV=1
Number of features: 0
Seq('MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIPDKFFIQLK...FIY')
Sequence ID: P20536
Sequence length: 218


In [4]:
#불러온 정보 쪼개기
from Bio import SeqIO

# Open the FASTA file
fasta_sequences = SeqIO.parse(open(seq_file),'fasta')
for fasta in fasta_sequences:
    print(fasta)
    print("----------------------")
    print("Seqence ID:", fasta.id)
    print("----------------------")
    print("Seqence name:", fasta.name)
    print("----------------------")
    fasta.description = fasta.description.split(" ")
    #seqence 설명 된 부분 쪼개기. 특성을 얻고잫 하였으나 각 seq마다 위치가 달라서 가능할지 의문
    #이걸 쪼개서 dataframe에 넣고 분류 기준으로 잡으면 좋을것 같다.
    print("Seqence description:", fasta.description[1:])
    print("----------------------")    
    print("Seqence:", repr(fasta.seq))
    print("----------------------")  
    print(fasta.annotations)
    break

ID: P20536
Name: P20536
Description: P20536 sp|P20536|UNG_VACCC Uracil-DNA glycosylase OS=Vaccinia virus (strain Copenhagen) OX=10249 GN=UNG PE=1 SV=1
Number of features: 0
Seq('MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIPDKFFIQLK...FIY')
----------------------
Seqence ID: P20536
----------------------
Seqence name: P20536
----------------------
Seqence description: ['sp|P20536|UNG_VACCC', 'Uracil-DNA', 'glycosylase', 'OS=Vaccinia', 'virus', '(strain', 'Copenhagen)', 'OX=10249', 'GN=UNG', 'PE=1', 'SV=1']
----------------------
Seqence: Seq('MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIPDKFFIQLK...FIY')
----------------------
{}


In [5]:
seq_df = data.drop(labels = ['sequence_length'], axis = 1, inplace = False)
#id 및 seqence만 별도 출력
#글자 사이 공백제거
seq_df

,id,sequence
0,P20536,M N S V T V S H A P Y T I T Y H D D W E P V M ...
1,O73864,M T E Y R N F L L L F I T S L S V I Y P C T G ...
2,O95231,M R L S S S P P R G P Q Q L S S F G S V D W L ...
3,A0A0B4J1F4,M G G E A G A D G P R G R V K S L G L V F E D ...
4,P54366,M V E T N S P P A G Y T L K R S P S D L G E Q ...
...,...,...
142241,A0A286YAI0,M E T E V D D F P G K A S I F S Q V N P L Y S ...
142242,A0A1D5NUC4,M S A A A S A E M I E T P P V L N F E E I D Y ...
142243,Q5RGB0,M A D K G P I L T S V I I F Y L S I G A A I F ...
142244,A0A2R8QMZ5,M G R K K I Q I T R I M D E R N R Q V T F T K ...


In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
term_file = "/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv"
term_df = pd.read_csv(term_file, delimiter = '\t')
term_df = term_df.rename(columns = {'EntryID':'id'})
term_df

,id,term,aspect
0,A0A009IHW8,GO:0008152,BPO
1,A0A009IHW8,GO:0034655,BPO
2,A0A009IHW8,GO:0072523,BPO
3,A0A009IHW8,GO:0044270,BPO
4,A0A009IHW8,GO:0006753,BPO
...,...,...,...
5363858,X5L565,GO:0050649,MFO
5363859,X5L565,GO:0016491,MFO
5363860,X5M5N0,GO:0005515,MFO
5363861,X5M5N0,GO:0005488,MFO


In [7]:
term_df['term'].value_counts().head(10)

GO:0005575    92912
GO:0008150    92210
GO:0110165    91286
GO:0003674    78637
GO:0005622    70785
GO:0009987    61293
GO:0043226    60883
GO:0043229    58315
GO:0005488    57380
GO:0043227    55452
Name: term, dtype: int64

In [ ]:
term_df[term_df['term']=='GO:0008152']

In [13]:
taxonomy_file = "/kaggle/input/cafa-5-protein-function-prediction/Train/train_taxonomy.tsv"
taxonomy_df = pd.read_csv(taxonomy_file, delimiter = '\t')
taxonomy_df = taxonomy_df.rename(columns = {'EntryID':'id'})
taxonomy_df

,id,taxonomyID
0,Q8IXT2,9606
1,Q04418,559292
2,A8DYA3,7227
3,Q9UUI3,284812
4,Q57ZS4,185431
...,...,...
142241,Q5TD07,9606
142242,A8BB17,7955
142243,A0A2R8QBB1,7955
142244,P0CT72,284812


데이터 합치기

In [10]:
df1 = term_df
df2 = seq_df

# df1과 df2를 id를 기준으로 합치기
merged_df = df1.merge(df2, on='id')

# 같은 id와 seq를 가진 행들의 function 합치기
merged_df = merged_df.groupby(['id', 'sequence','aspect'])['term'].apply(lambda x: '/'.join(x)).reset_index()

term_list = merged_df['term'].str.split('/')
for j in range(len(term_list[0])):
    column_name = 'term' + str(j+1)
    merged_df[column_name] = [terms[j] if len(terms) > j else None for terms in term_list]
    
merged_df = merged_df.drop(columns=['term'])
merged_df

,id,sequence,aspect,term1,term2,term3,term4,term5,term6,term7,...,term34,term35,term36,term37,term38,term39,term40,term41,term42,term43
0,A0A009IHW8,M S L E Q K K G A D I I S K I L Q I Q N S I G ...,BPO,GO:0008152,GO:0034655,GO:0072523,GO:0044270,GO:0006753,GO:1901292,GO:0044237,...,GO:1901575,GO:0072526,GO:0046434,GO:0009166,GO:0072524,GO:0006195,GO:0009056,GO:0044238,GO:0006793,GO:0019674
1,A0A009IHW8,M S L E Q K K G A D I I S K I L Q I Q N S I G ...,MFO,GO:0003674,GO:0003953,GO:0016787,GO:0016799,GO:0016798,GO:0003824,None,...,None,None,None,None,None,None,None,None,None,None
2,A0A021WW32,M F Y E H I I L A K K G P L A R I W L A A H W ...,BPO,GO:0048869,GO:0048856,GO:0022008,GO:0065007,GO:0007275,GO:0007059,GO:0098813,...,GO:0010628,GO:0021700,GO:0007064,GO:0048285,GO:0006996,GO:0000070,GO:0000278,GO:0030182,GO:0032501,GO:0010468
3,A0A021WW32,M F Y E H I I L A K K G P L A R I W L A A H W ...,CCO,GO:0099086,GO:0000228,GO:0005622,GO:0043226,GO:0000792,GO:0000795,GO:0000785,...,None,None,None,None,None,None,None,None,None,None
4,A0A021WZA4,M K Y I N C T Q P A I D D F P R D L F S E A Q ...,CCO,GO:0071944,GO:0005575,GO:0110165,GO:0016020,GO:0005886,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263754,X6RKS3,M A N L G C W M L V L F V A T W S D L G L C K ...,CCO,GO:0005622,GO:0031090,GO:0043226,GO:0031967,GO:0031965,GO:0031975,GO:0005575,...,None,None,None,None,None,None,None,None,None,None
263755,X6RLN4,E V K G L F K S E N C P K V I S C E F A H N S ...,CCO,GO:0005737,GO:0005829,GO:0005575,GO:0005622,GO:0110165,None,None,...,None,None,None,None,None,None,None,None,None,None
263756,X6RLP6,M A A P E Q P L A I S R G C T S S S S L S P P ...,CCO,GO:0005622,GO:0031981,GO:0043229,GO:0043226,GO:0110165,GO:0043231,GO:0070013,...,None,None,None,None,None,None,None,None,None,None
263757,X6RLR1,M A G L T D L Q R L Q A R V E E L E R W V Y G ...,CCO,GO:0005622,GO:0031981,GO:0043226,GO:0005730,GO:0043233,GO:0005575,GO:0043231,...,None,None,None,None,None,None,None,None,None,None


In [11]:
merged_df.columns

Index(['id', 'sequence', 'aspect', 'term1', 'term2', 'term3', 'term4', 'term5',
       'term6', 'term7', 'term8', 'term9', 'term10', 'term11', 'term12',
       'term13', 'term14', 'term15', 'term16', 'term17', 'term18', 'term19',
       'term20', 'term21', 'term22', 'term23', 'term24', 'term25', 'term26',
       'term27', 'term28', 'term29', 'term30', 'term31', 'term32', 'term33',
       'term34', 'term35', 'term36', 'term37', 'term38', 'term39', 'term40',
       'term41', 'term42', 'term43'],
      dtype='object')

In [14]:
merged_df2 = pd.merge(merged_df, taxonomy_df, on='id')
new_columns = ['id', 'taxonomyID', 'aspect', 'sequence', 'term1', 'term2', 'term3', 'term4', 'term5', 'term6', 'term7', 'term8', 'term9', 'term10', 'term11', 'term12', 'term13', 'term14', 'term15', 'term16', 'term17', 'term18', 'term19', 'term20', 'term21', 'term22', 'term23', 'term24', 'term25', 'term26', 'term27', 'term28', 'term29', 'term30', 'term31', 'term32', 'term33', 'term34', 'term35', 'term36', 'term37', 'term38', 'term39', 'term40', 'term41', 'term42', 'term43', 'term44', 'term45', 'term46', 'term47', 'term48', 'term49']
merged_df2 = merged_df2.reindex(columns=new_columns)
merged_df2

,id,taxonomyID,aspect,sequence,term1,term2,term3,term4,term5,term6,...,term40,term41,term42,term43,term44,term45,term46,term47,term48,term49
0,A0A009IHW8,1310613,BPO,M S L E Q K K G A D I I S K I L Q I Q N S I G ...,GO:0008152,GO:0034655,GO:0072523,GO:0044270,GO:0006753,GO:1901292,...,GO:0009056,GO:0044238,GO:0006793,GO:0019674,NaN,NaN,NaN,NaN,NaN,NaN
1,A0A009IHW8,1310613,MFO,M S L E Q K K G A D I I S K I L Q I Q N S I G ...,GO:0003674,GO:0003953,GO:0016787,GO:0016799,GO:0016798,GO:0003824,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
2,A0A021WW32,7227,BPO,M F Y E H I I L A K K G P L A R I W L A A H W ...,GO:0048869,GO:0048856,GO:0022008,GO:0065007,GO:0007275,GO:0007059,...,GO:0000278,GO:0030182,GO:0032501,GO:0010468,NaN,NaN,NaN,NaN,NaN,NaN
3,A0A021WW32,7227,CCO,M F Y E H I I L A K K G P L A R I W L A A H W ...,GO:0099086,GO:0000228,GO:0005622,GO:0043226,GO:0000792,GO:0000795,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
4,A0A021WZA4,7227,CCO,M K Y I N C T Q P A I D D F P R D L F S E A Q ...,GO:0071944,GO:0005575,GO:0110165,GO:0016020,GO:0005886,None,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263754,X6RKS3,9606,CCO,M A N L G C W M L V L F V A T W S D L G L C K ...,GO:0005622,GO:0031090,GO:0043226,GO:0031967,GO:0031965,GO:0031975,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
263755,X6RLN4,9606,CCO,E V K G L F K S E N C P K V I S C E F A H N S ...,GO:0005737,GO:0005829,GO:0005575,GO:0005622,GO:0110165,None,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
263756,X6RLP6,9606,CCO,M A A P E Q P L A I S R G C T S S S S L S P P ...,GO:0005622,GO:0031981,GO:0043229,GO:0043226,GO:0110165,GO:0043231,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
263757,X6RLR1,9606,CCO,M A G L T D L Q R L Q A R V E E L E R W V Y G ...,GO:0005622,GO:0031981,GO:0043226,GO:0005730,GO:0043233,GO:0005575,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
merged_df2.to_csv('id_taxid_aspect_seq_term.csv', index = False)

In [ ]:
import pandas as pd

df_max = pd.read_csv('/kaggle/input/2022-06-22/id_seq_term_tax.csv')
df_test = df_max[df_max['term1'] == 'GO:0008152'].drop(columns = ['term2', 'term3', 'term4', 'term5', 'term6', 'term7', 'term8', 'term9', 'term10', 'term11', 'term12', 'term13', 'term14', 'term15', 'term16', 'term17', 'term18', 'term19', 'term20', 'term21', 'term22', 'term23', 'term24', 'term25', 'term26', 'term27', 'term28', 'term29', 'term30', 'term31', 'term32', 'term33', 'term34', 'term35', 'term36', 'term37', 'term38', 'term39', 'term40', 'term41', 'term42', 'term43', 'term44', 'term45', 'term46', 'term47', 'term48', 'term49'])
df_test

In [ ]:
from nltk import FreqDist
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

#아미노산 단어 갯수 측정 함수
def find_common(amino_acid_sequences):
    # 아미노산 배열을 단어로 토큰화
    tokens = [word_tokenize(seq) for seq in amino_acid_sequences]

    # 모든 아미노산 배열의 단어들을 하나의 리스트로 통합
    all_words = [word for seq in tokens for word in seq]

    # 단어들의 빈도수 계산
    freq_dist = FreqDist(all_words)

    # 가장 빈도가 높은 상위 단어들을 반환
    common_patterns = freq_dist.most_common(10)

    return common_patterns

#아미노산 패턴 확인 함수
def find_common_patterns(amino_acid_sequences, min_n, max_n, min_frequency):
    # 아미노산 배열을 단어로 토큰화
    tokens = [word_tokenize(seq) for seq in amino_acid_sequences]

    # 모든 아미노산 배열의 단어들을 하나의 리스트로 통합
    all_words = [word for seq in tokens for word in seq]

    common_patterns = []
    for n in range(min_n, max_n + 1):
        # n-gram 생성
        ngram_list = list(ngrams(all_words, n))

        # n-gram 빈도수 계산
        freq_dist = FreqDist(ngram_list)

        # 가장 빈도가 높은 상위 n-gram들 중 발견 횟수가 min_frequency 이상인 것들을 추가
        common_patterns.extend([(pattern, count) for pattern, count in freq_dist.most_common(10) if count >= min_frequency])

    return common_patterns

In [ ]:
amino_acid_sequences = df_test['sequence']

In [ ]:
# 2개 이상 이어진 패턴 찾기 (n-gram의 최소값: 2, 최대값: 5)
common_patterns = find_common_patterns(amino_acid_sequences, 2, 5, 2000)
#min_frequency를 2000으로 설정 = 발견 횟수가 min_frequency 이상인 패턴들만 반환
#2는 n-gram의 크기를 나타내는 매개변수입니다. n-gram은 연속된 단어의 시퀀스로 구성된 패턴을 의미합니다. 
#위 코드의 경우 2로 설정하면 바이그램(bigram)을 생성하게 됩니다.

# 결과 출력
if common_patterns:
    print("Common patterns found:")
    for pattern, count in common_patterns:
        print(f"Pattern: {pattern}, Count: {count}")
else:
    print("No common patterns found.")

In [ ]:
amino_acid_sequences.info()

In [ ]:
len(amino_acid_sequences)

In [ ]:
import pandas as pd
import numpy as np

df_max = pd.read_csv('/kaggle/input/2022-06-22/id_seq_term_tax.csv')

# term에 'x'가 들어간 얘들 모두 찾는 코드
def find_seq(x):
    term_columns = ['term1', 'term2', 'term3', 'term4', 'term5', 'term6', 'term7', 'term8', 'term9', 'term10', 'term11', 'term12', 'term13', 'term14', 'term15', 'term16', 'term17', 'term18', 'term19', 'term20', 'term21', 'term22', 'term23', 'term24', 'term25', 'term26', 'term27', 'term28', 'term29', 'term30', 'term31', 'term32', 'term33', 'term34', 'term35', 'term36', 'term37', 'term38', 'term39', 'term40', 'term41', 'term42', 'term43', 'term44', 'term45', 'term46', 'term47', 'term48', 'term49']
    df_find = pd.DataFrame()
    for i in term_columns:
        df_find = pd.concat([df_find, df_max[df_max[i] == x]])    
    df_find.drop(columns = term_columns, axis=1, inplace=True)
    return df_find

def find_same_seq(x):
    amino_acid_sequences = x['sequence']
    for i in range(len(amino_acid_sequences)):
        if i == 0:
            string1 = amino_acid_sequences.values[i]
            string2 = amino_acid_sequences.values[i+1]
            output = ''.join([char1 if char1 == char2 else '-' for char1, char2 in zip(string1, string2)])
    
        elif i > 0 | i+1 < k:
            tring1 = output
            string2 = amino_acid_sequences.values[i+1]
            output = ''.join([char1 if char1 == char2 else '-' for char1, char2 in zip(string1, string2)])
    return output

result = find_seq('GO:0008152')
same_seq= find_same_seq(result)
print(same_seq)

1. GO:0005575    92912
2. GO:0008150    92210
3. GO:0110165    91286
4. GO:0003674    78637
5. GO:0005622    70785
6. GO:0009987    61293
7. GO:0043226    60883
8. GO:0043229    58315
9. GO:0005488    57380
10. GO:0043227    55452

In [ ]:
result = find_seq('GO:0005575')
same_seq= find_same_seq(result)
print(same_seq)

In [ ]:
result2 = find_seq('GO:0008150')
same_seq2= find_same_seq(result2)
print(same_seq2)

In [ ]:
result3 = find_seq('GO:0110165')
same_seq3= find_same_seq(result3)
print(same_seq3)

In [ ]:
result4 = find_seq('GO:0003674')
same_seq4= find_same_seq(result4)
print(same_seq4)

위는 직접 실행 해본 코드

In [ ]:
#차라리 term이름만 넣으면 공통 amino acide 가 나오도록 def만 만들까?

!단백질 시퀀스 정보를 벡터로 변환하는 방법(임베딩)에 대한 소개와 연구 동향
에서 소개된 embeding 방식
https://github.com/nadavbra/protein_bert/tree/master/proteinbert
https://github.com/mikejhuang/PhageProtVec

ProtBERT Vector Embeddings for Protein Sequences

In [ ]:
MAIN_DIR = "/kaggle/input/cafa-5-protein-function-prediction"

# UTILITARIES
import numpy as np
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# TORCH MODULES FOR METRICS COMPUTATION :
import torch
from torch.utils.data import Dataset
from torch import nn
from torch.utils.data import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchmetrics.classification import MultilabelF1Score
from torchmetrics.classification import MultilabelAccuracy

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger

# WANDB FOR LIGHTNING :
import wandb

# FILES VISUALIZATION
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
class config:
    train_sequences_path = MAIN_DIR  + "/Train/train_sequences.fasta"
    train_labels_path = MAIN_DIR + "/Train/train_terms.tsv"
    test_sequences_path = MAIN_DIR + "/Test (Targets)/testsuperset.fasta"
    
    num_labels = 500
    n_epochs = 5
    batch_size = 128
    lr = 0.001
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print("Load ProtBERT Model...")
# PROT BERT LOADING :
from Bio import SeqIO
from transformers import BertModel, BertTokenizer
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
model = BertModel.from_pretrained("Rostlab/prot_bert").to(config.device)

def get_bert_embedding(
    sequence : str,
    len_seq_limit : int
):
    '''
    Function to collect last hidden state embedding vector from pre-trained ProtBERT Model

    INPUTS:
    - sequence (str) : protein sequence (ex : AAABBB) from fasta file
    - len_seq_limit (int) : maximum sequence lenght (i.e nb of letters) for truncation

    OUTPUTS:
    - output_hidden : last hidden state embedding vector for input sequence of length 1024
    '''
    sequence_w_spaces = ' '.join(list(sequence))
    encoded_input = tokenizer(
        sequence_w_spaces,
        truncation=True,
        max_length=len_seq_limit,
        padding='max_length',
        return_tensors='pt').to(config.device)
    output = model(**encoded_input)
    output_hidden = output['last_hidden_state'][:,0][0].detach().cpu().numpy()
    assert len(output_hidden)==1024
    return output_hidden

seq_file = "/kaggle/input/cafa-5-protein-function-prediction/Train/train_sequences.fasta"

### COLLECTING FOR TRAIN SAMPLES :
print("Loading train set ProtBERT Embeddings...")
fasta_train = SeqIO.parse(open(seq_file), "fasta")
print("Total Nb of Elements : ", len(list(fasta_train)))
fasta_train = SeqIO.parse(open(seq_file), "fasta")
ids_list = []
embed_vects_list = []
t0 = time.time()
checkpoint = 0
for item in tqdm(fasta_train):
    ids_list.append(item.id)
    embed_vects_list.append(
        get_bert_embedding(sequence = item.seq, len_seq_limit = 1200))
    checkpoint+=1
    if checkpoint>=100:
        df_res = pd.DataFrame(data={"id" : ids_list, "embed_vect" : embed_vects_list})
        np.save('/kaggle/working/train_ids.npy',np.array(ids_list))
        np.save('/kaggle/working/train_embeddings.npy',np.array(embed_vects_list))
        checkpoint=0

np.save('/kaggle/working/train_ids.npy',np.array(ids_list))
np.save('/kaggle/working/train_embeddings.npy',np.array(embed_vects_list))
print('Total Elapsed Time:',time.time()-t0)

### COLLECTING FOR TEST SAMPLES :
print("Loading test set ProtBERT Embeddings...")
fasta_test = SeqIO.parse(config.test_sequences_path, "fasta")
print("Total Nb of Elements : ", len(list(fasta_test)))
fasta_test = SeqIO.parse(config.test_sequences_path, "fasta")
ids_list = []
embed_vects_list = []
t0 = time.time()
checkpoint=0
for item in tqdm(fasta_test):
    ids_list.append(item.id)
    embed_vects_list.append(
        get_bert_embedding(sequence = item.seq, len_seq_limit = 1200))
    checkpoint+=1
    if checkpoint>=100:
        np.save('/kaggle/working/test_ids.npy',np.array(ids_list))
        np.save('/kaggle/working/test_embeddings.npy',np.array(embed_vects_list))
        checkpoint=0

np.save('/kaggle/working/test_ids.npy',np.array(ids_list))
np.save('/kaggle/working/test_embeddings.npy',np.array(embed_vects_list))
print('Total Elasped Time:',time.time()-t0)


위 모델은 embeding test 중

In [ ]:
from __future__ import print_function,division

import sys
import numpy as np
import h5py

import torch
import torch.nn as nn
import torch.nn.functional as F

from src.alphabets import Uniprot21
import src.fasta as fasta
import src.models.sequence


def unstack_lstm(lstm):
    device = next(iter(lstm.parameters())).device

    in_size = lstm.input_size
    hidden_dim = lstm.hidden_size
    layers = []
    for i in range(lstm.num_layers):
        layer = nn.LSTM(in_size, hidden_dim, batch_first=True, bidirectional=True)
        layer.to(device)

        attributes = ['weight_ih_l', 'weight_hh_l', 'bias_ih_l', 'bias_hh_l']
        for attr in attributes:
            dest = attr + '0'
            src = attr + str(i)
            getattr(layer, dest).data[:] = getattr(lstm, src)
            #setattr(layer, dest, getattr(lstm, src))
            
            dest = attr + '0_reverse'
            src = attr + str(i) + '_reverse'
            getattr(layer, dest).data[:] = getattr(lstm, src)
            #setattr(layer, dest, getattr(lstm, src))
        layer.flatten_parameters()
        layers.append(layer)
        in_size = 2*hidden_dim
    return layers

def embed_stack(x, lm_embed, lstm_stack, proj, include_lm=True, final_only=False):
    zs = []
    
    x_onehot = x.new(x.size(0),x.size(1), 21).float().zero_()
    x_onehot.scatter_(2,x.unsqueeze(2),1)
    zs.append(x_onehot)
    
    h = lm_embed(x)
    if include_lm and not final_only:
        zs.append(h)

    if lstm_stack is not None:
        for lstm in lstm_stack:
            h,_ = lstm(h)
            if not final_only:
                zs.append(h)
        h = proj(h.squeeze(0)).unsqueeze(0)
        zs.append(h)

    z = torch.cat(zs, 2)
    return z


def embed_sequence(x, lm_embed, lstm_stack, proj, include_lm=True, final_only=False
                  ,  pool='none', use_cuda=False):

    if len(x) == 0:
        return None

    alphabet = Uniprot21()
    x = x.upper()
    # convert to alphabet index
    x = alphabet.encode(x)
    x = torch.from_numpy(x)
    if use_cuda:
        x = x.cuda()

    # embed the sequence
    with torch.no_grad():
        x = x.long().unsqueeze(0)
        z = embed_stack(x, lm_embed, lstm_stack, proj
                       , include_lm=include_lm, final_only=final_only)
        # pool if needed
        z = z.squeeze(0)
        if pool == 'sum':
            z = z.sum(0)
        elif pool == 'max':
            z,_ = z.max(0)
        elif pool == 'avg':
            z = z.mean(0)
        z = z.cpu().numpy()

    return z


def load_model(path, use_cuda=False):
    encoder = torch.load(path)
    encoder.eval()

    if use_cuda:
        encoder.cuda()

    if type(encoder) is src.models.sequence.BiLM:
        # model is only the LM
        return encoder.encode, None, None

    encoder = encoder.embedding

    lm_embed = encoder.embed
    lstm_stack = unstack_lstm(encoder.rnn)
    proj = encoder.proj

    return lm_embed, lstm_stack, proj


def main():
    import argparse
    parser = argparse.ArgumentParser('Script for embedding fasta format sequences using a saved embedding model. Saves embeddings as HDF5 file.')

    parser.add_argument('path', help='sequences to embed in fasta format')
    parser.add_argument('-m', '--model', help='path to saved embedding model')
    parser.add_argument('-o', '--output', help='path to HDF5 output file')
    parser.add_argument('--lm-only', action='store_true', help='only return the language model hidden layers')
    parser.add_argument('--no-lm', action='store_true', help='do not include LM hidden layers in embedding. by default, all hidden layers of all layers are concatenated and returned by this script.')
    parser.add_argument('--proj-only', action='store_true', help='only return the final structure-learned embedding')
    parser.add_argument('--pool', choices=['none', 'sum', 'max', 'avg'], default='none', help='apply some pooling operation over each sequence (default: none)')
    parser.add_argument('-d', '--device', type=int, default=-2, help='compute device to use')

    args = parser.parse_args()

    path = args.path

    # set the device
    d = args.device
    use_cuda = (d != -1) and torch.cuda.is_available()
    if d >= 0:
        torch.cuda.set_device(d)

    # load the model
    lm_embed, lstm_stack, proj = load_model(args.model, use_cuda=use_cuda)

    # parse the sequences and embed them
    # write them to hdf5 file
    print('# writing:', args.output, file=sys.stderr)
    h5 = h5py.File(args.output, 'w')

    lm_only = args.lm_only
    if lm_only:
        lstm_stack = None
        proj = None

    no_lm = args.no_lm
    include_lm = not no_lm
    final_only = args.proj_only

    pool = args.pool
    print('# embedding with lm_only={}, no_lm={}, proj_only={}'.format(lm_only, no_lm, final_only), file=sys.stderr)
    print('# pooling:', pool, file=sys.stderr)

    count = 0
    with open(path, 'rb') as f:
        for name,sequence in fasta.parse_stream(f):
            # use sequence name as HDF key
            pid = name.decode('utf-8')
            if len(sequence) == 0:
                print('# WARNING: sequence', pid, 'has length=0. Skipping.', file=sys.stderr)
                continue
            # only do pids we haven't done already...
            if pid not in h5:
                z = embed_sequence(sequence, lm_embed, lstm_stack, proj
                                  , include_lm=include_lm, final_only=final_only
                                  , pool=pool, use_cuda=use_cuda)
                # write as hdf5 dataset
                h5.create_dataset(pid, data=z, compression='lzf')
            count += 1
            print('# {} sequences processed...'.format(count), file=sys.stderr, end='\r')
    print(' '*80, file=sys.stderr, end='\r')
    print('# Done!', file=sys.stderr)



if __name__ == '__main__':
    main()





In [ ]:
#seq_file에서 dataframe 형식으로 data 불러오기
seq_file = "/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset.fasta"

In [ ]:
import pandas as pd
import re
def read_fasta(file_path, columns) :
    from Bio.SeqIO.FastaIO import SimpleFastaParser 
    with open(file_path) as fasta_file :  
        records = [] # create empty list

        for title, sequence in SimpleFastaParser(fasta_file):  #SimpleFastaParser Iterate over Fasta records as string tuples. For each record a tuple of two strings is returned: the FASTA title line (without the leading ‘>’ character),  and the sequence (with any whitespace removed). 
            record = []
            title_splits=re.findall(r"[\w']+", title) # Data cleaning is needed
            record.append(title_splits[0])  #First values are ID (Append adds element to a list)
            record.append(len(sequence)) #Second values are sequences lengths
            sequence = " ".join(sequence) #It converts into one line
            record.append(sequence)#Third values are sequences
            records.append(record)
    return pd.DataFrame(records, columns = columns) #We have created a function that returns a dataframe

#Now let's use this function by inserting in the first argument the file name (or file path if your working directory is different from where the fasta file is)        
#And in the second one the names of columns
data = read_fasta(seq_file, columns=["id","sequence_length", "sequence"])
data
var_seq = data.loc[0,'sequence']

In [ ]:
#불러온 정보 쪼개기
from Bio import SeqIO

# Open the FASTA file
fasta_sequences = SeqIO.parse(open(seq_file),'fasta')
for fasta in fasta_sequences:
    print(fasta)
    print("----------------------")
    print("Seqence ID:", fasta.id)
    print("----------------------")
    print("Seqence name:", fasta.name)
    print("----------------------")
    fasta.description = fasta.description.split(" ")
    #seqence 설명 된 부분 쪼개기. 특성을 얻고잫 하였으나 각 seq마다 위치가 달라서 가능할지 의문
    #이걸 쪼개서 dataframe에 넣고 분류 기준으로 잡으면 좋을것 같다.
    print("Seqence description:", fasta.description[1:])
    print("----------------------")    
    print("Seqence:", repr(fasta.seq))
    print("----------------------")  
    print(fasta.annotations)
    break

In [ ]:
var_seq

In [ ]:
ALL_AAS = 'ACDEFGHIKLMNPQRSTUVWXY'
ADDITIONAL_TOKENS = ['<OTHER>', '<START>', '<END>', '<PAD>']

# Each sequence is added <START> and <END> tokens
ADDED_TOKENS_PER_SEQ = 2

n_aas = len(ALL_AAS)
aa_to_token_index = {aa: i for i, aa in enumerate(ALL_AAS)}
additional_token_to_index = {token: i + n_aas for i, token in enumerate(ADDITIONAL_TOKENS)}
token_to_index = {**aa_to_token_index, **additional_token_to_index}
index_to_token = {index: token for token, index in token_to_index.items()}
n_tokens = len(token_to_index)

def tokenize_seq(seq):
    other_token_index = additional_token_to_index['<OTHER>']
    return [additional_token_to_index['<START>']] + [aa_to_token_index.get(aa, other_token_index) for aa in parse_seq(seq)] + \
            [additional_token_to_index['<END>']]
            
def parse_seq(seq):
    if isinstance(seq, str):
        return seq
    elif isinstance(seq, bytes):
        return seq.decode('utf8')
    else:
        raise TypeError('Unexpected sequence type: %s' % type(seq))
        
tokenize_seq(var_seq)
# parse_seq(var_seq)

https://github.com/murhekar/aa-vec-embeddings/blob/master/scripts/models.py
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6061698/
https://www.kaggle.com/code/danofer/deep-protein-sequence-family-classification/notebook
https://www.kaggle.com/code/basu369victor/attention-based-protein-structure-prediction